In [2]:
#Utility
import urllib
import os
import datetime

#Data Science
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm

#Google API
#import google.auth
#from google.cloud import bigquery
#from google.cloud import bigquery_storage_v1beta2

In [ ]:
#ACCOUNT IS SUSPENDED AT THE MOMENT ----------------------------------------------------
#Big query, requires API json 
#pip install --upgrade google-cloud-bigquery[bqstorage,pandas]
#pip install --upgrade google-auth
#%load_ext google.cloud.bigquery
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="Covid-19-be0adf05131c.json"

#%%bigquery jhu_summary
#SELECT * FROM `bigquery-public-data.covid19_jhu_csse.summary`
#jhu_summary['country_region'].value_counts()

## Mobility

In [3]:
#Import mobility from url
url = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'
mobility = pd.read_csv(url)

#Transform mobility to time series
startdate = datetime.datetime.strptime('2020-02-15','%Y-%m-%d').date()
enddate = datetime.datetime.strptime('2020-04-11','%Y-%m-%d').date()

filterlist = ['retail_and_recreation_percent_change_from_baseline',
'grocery_and_pharmacy_percent_change_from_baseline',
'parks_percent_change_from_baseline',
'transit_stations_percent_change_from_baseline',
'workplaces_percent_change_from_baseline',
'residential_percent_change_from_baseline']

clist = ['retail_rec_mobility','grocery_pharmacy_mobility','parks_mobility','transit_mobility','workplaces_mobility','residential_mobility']

C:\Users\lueda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
country = mobility['country_region'].value_counts()
countrylist = country.index.to_list()
usmobility = mobility[mobility['country_region']=='United States']

#Change column names
usmobility = usmobility.rename(columns={"sub_region_1":"Province_State", "sub_region_2": "Admin2"})

#Split county text out of county
countylist = []
txt = usmobility['Admin2'].to_list()
for i in txt:
    i = str(i)
    countylist.append(i.replace(' County', ''))
usmobility['Admin2'] = countylist

usmobility['Lookup'] = usmobility['Admin2'] + usmobility['Province_State'] + usmobility['date']
lookuplist = usmobility['Lookup'].to_list()
usmobility = usmobility.transpose()
usmobility.columns = lookuplist
usmobility = usmobility.transpose()

usmobility.loc[:,filterlist].astype(float)
usmobility = usmobility.loc[:,filterlist]

## Cases from John Hopkins

In [5]:
#Import confirmed cases from url
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

In [6]:
df = confirmed.copy()
df.columns
xlist = ['UID', 'iso2', 'iso3', 'code3', 'FIPS','Country_Region', 'Lat', 'Long_','Combined_Key']

df['Lookup'] = df['Admin2'] + df['Province_State']
includelist = [x for x in df.columns if x not in xlist]
datelist = includelist.copy()
datelist.pop(0)
datelist.pop(0)
datelist.remove('Lookup')
df = df.loc[:,includelist]

lookuplist = df['Lookup'].to_list()
df = df.transpose()
df.columns = lookuplist
df = df.transpose()
lookuplist = df['Lookup'].value_counts().index.to_list()
lookuplist = sorted(lookuplist)

## Contructing Main Dataframe

In [7]:
#Construction of maindf
colnames = ['Lookup','State','County','Date','Confirmed','retail_and_recreation_percent_change_from_baseline',
'grocery_and_pharmacy_percent_change_from_baseline',
'parks_percent_change_from_baseline',
'transit_stations_percent_change_from_baseline',
'workplaces_percent_change_from_baseline',
'residential_percent_change_from_baseline']
maindf = pd.DataFrame(index=range(0,len(lookuplist)*len(datelist)), columns=colnames)

#Start loop for confirmed
j = 0
for i in lookuplist:
    testdf = df.loc[i,:]
    timeseries = testdf[2:-1]
    date = timeseries.index.to_list()
    timeseries = timeseries.to_list()
    length = len(timeseries)
    maindf.iloc[j:(j+length),colnames.index('Lookup')] = i
    maindf.iloc[j:(j+length),colnames.index('State')] = testdf[1]
    maindf.iloc[j:(j+length),colnames.index('County')] = testdf[0]
    maindf.iloc[j:(j+length),colnames.index('Date')] = date
    maindf.iloc[j:(j+length),colnames.index('Confirmed')] = timeseries 
    j = j+length
    
maindf.loc[:,'Date1'] = pd.to_datetime(maindf['Date'],format='%m/%d/%y')
#test = maindf['Date1'].astype(str).tolist()
maindf['Lookup'] = maindf['Lookup'] + maindf['Date1'].astype(str).tolist()

labels = maindf['Lookup'].to_list()
maindf = maindf.transpose()
maindf.columns = labels
maindf = maindf.transpose()

In [8]:
maindf.loc[:, "Confirmed"].value_counts()

0        222625
1         15565
2          8953
3          6723
4          5080
          ...  
1354          1
3206          1
1361          1
1363          1
32765         1
Name: Confirmed, Length: 3058, dtype: int64

In [16]:
maindf.loc[:, "state-county"] = maindf.loc[:, "State"] + "-" + maindf.loc[:, "County"]
maindf.to_csv("maindf.csv", index = False)